# Covid 19 EDA

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import random
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

#Color Palette
cnf="#393e46"
dth="#ff2e63"
rec="#21bf73"
act="#fe9801"

In [ ]:
import plotly as py
py.offline.init_notebook_mode(connected=True)   ## For running plotly offline

## Loading the data

In [ ]:
country_daywise=pd.read_csv("../Covid 19/Data/country_daywise.csv",parse_dates=["Date"])
countrywise=pd.read_csv("../Covid 19/Data/countrywise.csv")
daywise=pd.read_csv("../Covid 19/Data/daywise.csv",parse_dates=["Date"])


In [ ]:
data_cleaned=pd.read_csv("../Covid 19/Data/covid_19_data_cleaned.csv",parse_dates=["Date"])
data_cleaned.head()

In [ ]:
data_cleaned["Province/State"]=data_cleaned["Province/State"].fillna("")
data_cleaned.head()

In [ ]:
data_cleaned.info(memory_usage="deep")

from datacleaned getting confirmed

In [ ]:
confirmed=data_cleaned.groupby("Date").agg({"Confirmed":"sum"}).reset_index()
confirmed

In [ ]:
recovered=data_cleaned.groupby("Date").agg({"Recovered":"sum"}).reset_index()
recovered

In [ ]:
deaths=data_cleaned.groupby("Date").agg({"Deaths":"sum"}).reset_index()
deaths

In [ ]:
data_cleaned.isna().sum()

In [ ]:
data_cleaned.query("Country=='India'")

## Worldwide Total Confirmed, Recovered, Deaths

In [ ]:
confirmed.tail()

In [ ]:
recovered.tail()

In [ ]:
deaths.tail()

### Plotting

First we have to create a figure

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=confirmed["Date"],y=confirmed["Confirmed"],mode="lines+markers",name="Confirmed Cases",line={"color":"Orange","width":1}))
fig.add_trace(go.Scatter(x=recovered["Date"],y=recovered["Recovered"],mode="lines+markers",name="Recovered Cases",line={"color":"Green","width":1}))
fig.add_trace(go.Scatter(x=deaths["Date"],y=deaths["Deaths"],mode="lines+markers",name="Confirmed Cases",line={"color":"Red","width":1}))
fig.update_layout(title="Worldwide Covid-19 Cases",xaxis_tickfont_size=14,yaxis={"title":"No of Cases"})
fig.show()

## Case Density Animation on WorldMap

In [ ]:
data_cleaned.info()

To mark value in world map we have to convert date to str

In [ ]:
data_cleaned["Date"]=data_cleaned["Date"].astype("str")
data_cleaned.info()

### Plotting in world map

In [ ]:
fig=px.density_mapbox(data_frame=data_cleaned,lat="Lat",lon="Long",hover_name="Country",hover_data=["Confirmed","Recovered","Deaths"],animation_frame="Date",color_continuous_scale="Portland",zoom=0,radius=7,height=600)
fig.update_layout(title="Worldwide Covid-19 Cases Timelapse")
fig.update_layout(mapbox_style="open-street-map",mapbox_center_lon=0)
fig.show()

## Total Cases on Ships

In [ ]:
data_cleaned["Date"]=pd.to_datetime(data_cleaned["Date"])
data_cleaned.info(memory_usage="deep")

In [ ]:
ship_rows=data_cleaned["Province/State"].str.contains("Grand Princess")|data_cleaned["Province/State"].str.contains("Diamond Princess")|data_cleaned["Country"].str.contains("Grand Princess")|data_cleaned["Country"].str.contains("Diamond Princess")|data_cleaned["Country"].str.contains("MS Zaandam")
ship=data_cleaned[ship_rows]
ship

Lets remove this ships from data_cleaned

In [ ]:
data_cleaned=data_cleaned.drop(ship.index)

In [ ]:
data_cleaned

In [ ]:
ship_latest=ship[ship["Date"]==max(ship["Date"])]
ship_latest.style.background_gradient(cmap="Pastel1_r")

## Cases over Area Plot

In [ ]:
temp=data_cleaned.groupby("Date").agg({"Confirmed":"sum","Recovered":"sum","Deaths":"sum","Active":"sum"}).reset_index()
temp

In [ ]:
temp=temp.tail(1)
temp

In [ ]:
tm=temp.melt(id_vars="Date",value_vars=["Confirmed","Recovered","Deaths","Active"])
tm

In [ ]:
fig=px.treemap(tm,path=["variable"],values="value",height=250,width=800,color_discrete_sequence=[act,rec,dth])
fig.data[0].textinfo="label+text+value"
fig.show()

In [ ]:
temp=data_cleaned.groupby("Date").agg({"Confirmed":"sum","Recovered":"sum","Deaths":"sum","Active":"sum"}).reset_index()
temp

In [ ]:
temp=temp.melt(id_vars="Date",value_vars=["Recovered","Deaths","Active"],var_name="Case",value_name="Count")
temp

In [ ]:
fig=px.area(data_frame=temp,x='Date',y="Count",color="Case",height=600,title="Cases over time",color_discrete_sequence=[rec,dth,act])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

## Folium Maps

In [ ]:
temp=data_cleaned[data_cleaned["Date"]==max(data_cleaned["Date"])].reset_index()
temp

In [ ]:
m=folium.Map(location=[0,0],tiles="cartodbpositron",min_zoom=1,max_zoom=4,zoom_start=1)
for i in range(0,len(temp)):
    folium.Circle(location=(temp.loc[i,["Lat","Long"]]),color="crimson",fill="crimson",tooltip="<li><bold>Country: "+str(temp.loc[i,"Country"])+"<li><bold>Province: "+str(temp.loc[i,"Province/State"])+"<li><bold>Confirmed: "+str(temp.loc[i,"Confirmed"])+"<li><bold>Death: "+str(temp.loc[i,"Deaths"]),radius=int(temp.loc[i,"Confirmed"])**0.5).add_to(m)
m

## Confirmed Cases with Choropleth Map

In [ ]:
country_daywise

In [ ]:
fig=px.choropleth(data_frame=country_daywise,locations="Country",locationmode="country names",color=np.log(country_daywise["Confirmed"]),hover_name="Country",animation_frame=country_daywise["Date"].dt.strftime("%Y-%M-%D"),color_continuous_scale=px.colors.sequential.Inferno,title="Cases over Time")
fig.update(layout_coloraxis_showscale=True)
fig.show()

## Deaths and Recoveries per 100 Cases

In [ ]:
daywise.head()

In [ ]:
fig_c=px.bar(data_frame=daywise,x="Date",y="Confirmed",color_discrete_sequence=[act])
fig_d=px.bar(data_frame=daywise,x="Date",y="Deaths",color_discrete_sequence=[dth])
fig=make_subplots(rows=1,cols=2,shared_xaxes=False,horizontal_spacing=0.1,subplot_titles=("Confirmed Cases","Deaths Cases"))
fig.add_trace(fig_c["data"][0],row=1,col=1)
fig.add_trace(fig_d["data"][0],row=1,col=2)
fig.update_layout(height=480)
fig.show()

## Confirmed and Death Cases with Static Colormap

In [ ]:
fig_c=px.choropleth(data_frame=countrywise,locations="Country",locationmode="country names",color=np.log(countrywise["Confirmed"]),hover_name="Country",hover_data=["Confirmed"])
temp=countrywise[countrywise["Deaths"]>0]
fig_d=px.choropleth(data_frame=temp,locations="Country",locationmode="country names",color=np.log(temp["Deaths"]),hover_name="Country",hover_data=["Deaths"])
fig=make_subplots(rows=1,cols=2,subplot_titles=("Confirmed","Deaths"),specs=[[{"type":"choropleth"},{"type":"choropleth"}]])
fig.add_trace(fig_c["data"][0],row=1,col=1)
fig.add_trace(fig_d["data"][0],row=1,col=2)
fig.show()